# **Install**

In [1]:
import os
import requests
import json
import pandas as pd
import re
from concurrent.futures import ThreadPoolExecutor, as_completed

# **Data**

In [ ]:
DATA_PATH = "data"
RESULT_PATH = '<YOUR_PATH>'
BACKUP_PATH = os.path.join(RESULT_PATH, "raw_responses.jsonl")

# Assume the vllm is already on the background running with max model len 4096
API_URL  = "<API_HERE>"
MODEL = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"

# **Usage**

In [4]:
def ask_model(question: str):
    """
    Sends a request to the model server and fetches a response.
    """
    url = API_URL
    headers = {"Content-Type": "application/json"}
    data = {
        "model": MODEL,
        "messages": [
            {
                "role": "user",
                "content": question
            }
        ]
    }

    response = requests.post(url, headers=headers, json=data)
    response.raise_for_status()
    return response.json()

In [ ]:
# Usage:
result = ask_model("What is the capital of France?")
print(json.dumps(result, indent=2))

In [6]:
def process_row(row):
    prompt = (
        "<think>\n"
        f"Question: {row['question']}\n"
        f"Options: {row['options']}\n\n"
        "Choose the correct option and explain your reasoning. "
        "Please reason step by step, and put your final answer within \\boxed{}."
    )
    response = ask_model(prompt)

    msg   = response["choices"][0]["message"]
    cot   = msg.get("reasoning_content")
    raw   = msg.get("content")
    final = raw if isinstance(raw, str) else ""

    # now safe to regex
    boxed_m = re.search(r"\\boxed\{(.*?)\}", final)
    boxed   = boxed_m.group(1).strip() if boxed_m else None

    entry = row.to_dict()
    entry.update({
        "chain_of_thought":  cot,
        "final_answer":      final,
        "correct_response":  boxed
    })

    backup_entry = {
        "question_id": row["question_id"],
        "q_type":      row["q_type"],
        "response":    response
    }
    return entry, backup_entry

for filename in os.listdir(DATA_PATH):
    if not filename.endswith("_qa_df.pkl"):
        continue

    q_type = filename.replace("_qa_df.pkl", "")
    print(f"Processing file for q_type: {q_type}")
    df     = pd.read_pickle(os.path.join(DATA_PATH, filename))

    # 1) collect exactly 50 valid rows
    tasks = []
    for _, row in df.iterrows():
        if pd.isna(row["expected_answer"]):
            continue
        tasks.append(row)
        if len(tasks) == 50:
            break

    total     = len(tasks)
    responses = []

    with ThreadPoolExecutor(max_workers=4) as pool:
        futures = {pool.submit(process_row, row): row for row in tasks}

        completed = 0
        for fut in as_completed(futures):
            try:
                entry, backup_entry = fut.result()
            except Exception as e:
                qid = futures[fut]["question_id"]
                print(f"[{q_type}] ERROR on question_id {qid}: {e}")
                continue

            # write backup safely in main thread
            with open(BACKUP_PATH, "a", encoding="utf-8") as bf:
                bf.write(json.dumps(backup_entry, ensure_ascii=False) + "\n")

            responses.append(entry)
            completed += 1
            print(f"[{q_type}] Progress: {completed}/{total} ✓ question_id={entry['question_id']}")

    # 2) persist final DataFrame
    out_df   = pd.DataFrame(responses)
    csv_path = os.path.join(RESULT_PATH, f"responses_{q_type}.csv")
    out_df.to_csv(csv_path, index=False)
    print(f"Saved CSV for q_type '{q_type}' to {csv_path}")

Processing file for q_type: top_trading
[top_trading] Progress: 1/50 ✓ question_id=0_0
[top_trading] Progress: 2/50 ✓ question_id=15_0
[top_trading] Progress: 3/50 ✓ question_id=2_0
[top_trading] Progress: 4/50 ✓ question_id=1_0
[top_trading] Progress: 5/50 ✓ question_id=31_0
[top_trading] Progress: 6/50 ✓ question_id=21_0
[top_trading] Progress: 7/50 ✓ question_id=42_0
[top_trading] Progress: 8/50 ✓ question_id=68_0
[top_trading] Progress: 9/50 ✓ question_id=40_0
[top_trading] Progress: 10/50 ✓ question_id=71_0
[top_trading] Progress: 11/50 ✓ question_id=103_0
[top_trading] Progress: 12/50 ✓ question_id=174_0
[top_trading] Progress: 13/50 ✓ question_id=90_0
[top_trading] Progress: 14/50 ✓ question_id=127_0
[top_trading] Progress: 15/50 ✓ question_id=176_0
[top_trading] Progress: 16/50 ✓ question_id=141_0
[top_trading] Progress: 17/50 ✓ question_id=178_0
[top_trading] Progress: 18/50 ✓ question_id=185_0
[top_trading] Progress: 19/50 ✓ question_id=203_0
[top_trading] Progress: 20/50 ✓ q

[time_inconsistency] Progress: 4/50 ✓ question_id=1_0
[time_inconsistency] Progress: 5/50 ✓ question_id=4_0
[time_inconsistency] Progress: 6/50 ✓ question_id=6_0
[time_inconsistency] Progress: 7/50 ✓ question_id=5_0
[time_inconsistency] Progress: 8/50 ✓ question_id=7_0
[time_inconsistency] Progress: 9/50 ✓ question_id=9_0
[time_inconsistency] Progress: 10/50 ✓ question_id=8_0
[time_inconsistency] Progress: 11/50 ✓ question_id=10_0
[time_inconsistency] Progress: 12/50 ✓ question_id=11_0
[time_inconsistency] Progress: 13/50 ✓ question_id=12_0
[time_inconsistency] Progress: 14/50 ✓ question_id=13_0
[time_inconsistency] Progress: 15/50 ✓ question_id=14_0
[time_inconsistency] Progress: 16/50 ✓ question_id=15_0
[time_inconsistency] Progress: 17/50 ✓ question_id=17_0
[time_inconsistency] Progress: 18/50 ✓ question_id=16_0
[time_inconsistency] Progress: 19/50 ✓ question_id=18_0
[time_inconsistency] Progress: 20/50 ✓ question_id=19_0
[time_inconsistency] Progress: 21/50 ✓ question_id=20_0
[time

[prob_basic] Progress: 47/50 ✓ question_id=46_0
[prob_basic] Progress: 48/50 ✓ question_id=49_0
[prob_basic] Progress: 49/50 ✓ question_id=45_0
[prob_basic] Progress: 50/50 ✓ question_id=48_0
Saved CSV for q_type 'prob_basic' to results_ds_7b/responses_prob_basic.csv
Processing file for q_type: level_k
[level_k] Progress: 1/50 ✓ question_id=0_0
[level_k] Progress: 2/50 ✓ question_id=1_0
[level_k] Progress: 3/50 ✓ question_id=3_0
[level_k] Progress: 4/50 ✓ question_id=2_0
[level_k] Progress: 5/50 ✓ question_id=7_0
[level_k] Progress: 6/50 ✓ question_id=4_0
[level_k] Progress: 7/50 ✓ question_id=6_0
[level_k] Progress: 8/50 ✓ question_id=5_0
[level_k] Progress: 9/50 ✓ question_id=8_0
[level_k] Progress: 10/50 ✓ question_id=11_0
[level_k] Progress: 11/50 ✓ question_id=9_0
[level_k] Progress: 12/50 ✓ question_id=10_0
[level_k] Progress: 13/50 ✓ question_id=12_0
[level_k] Progress: 14/50 ✓ question_id=13_0
[level_k] Progress: 15/50 ✓ question_id=14_0
[level_k] Progress: 16/50 ✓ question_id=

[first_order_false_belief] Progress: 49/50 ✓ question_id=94_0
[first_order_false_belief] Progress: 50/50 ✓ question_id=82_0
Saved CSV for q_type 'first_order_false_belief' to results_ds_7b/responses_first_order_false_belief.csv
Processing file for q_type: plurality_voting
[plurality_voting] Progress: 1/50 ✓ question_id=1_0
[plurality_voting] Progress: 2/50 ✓ question_id=3_0
[plurality_voting] Progress: 3/50 ✓ question_id=0_0
[plurality_voting] Progress: 4/50 ✓ question_id=2_0
[plurality_voting] Progress: 5/50 ✓ question_id=4_0
[plurality_voting] Progress: 6/50 ✓ question_id=5_0
[plurality_voting] Progress: 7/50 ✓ question_id=6_0
[plurality_voting] Progress: 8/50 ✓ question_id=8_0
[plurality_voting] Progress: 9/50 ✓ question_id=7_0
[plurality_voting] Progress: 10/50 ✓ question_id=11_0
[plurality_voting] Progress: 11/50 ✓ question_id=10_0
[plurality_voting] Progress: 12/50 ✓ question_id=9_0
[plurality_voting] Progress: 13/50 ✓ question_id=12_0
[plurality_voting] Progress: 14/50 ✓ questio

[false_belief] Progress: 48/50 ✓ question_id=41_0
[false_belief] Progress: 49/50 ✓ question_id=48_0
[false_belief] Progress: 50/50 ✓ question_id=46_0
Saved CSV for q_type 'false_belief' to results_ds_7b/responses_false_belief.csv
Processing file for q_type: independence_risk
[independence_risk] Progress: 1/50 ✓ question_id=0_0
[independence_risk] Progress: 2/50 ✓ question_id=1_0
[independence_risk] Progress: 3/50 ✓ question_id=3_0
[independence_risk] Progress: 4/50 ✓ question_id=4_0
[independence_risk] Progress: 5/50 ✓ question_id=6_0
[independence_risk] Progress: 6/50 ✓ question_id=2_0
[independence_risk] Progress: 7/50 ✓ question_id=8_0
[independence_risk] Progress: 8/50 ✓ question_id=5_0
[independence_risk] Progress: 9/50 ✓ question_id=11_0
[independence_risk] Progress: 10/50 ✓ question_id=7_0
[independence_risk] Progress: 11/50 ✓ question_id=12_0
[independence_risk] Progress: 12/50 ✓ question_id=13_0
[independence_risk] Progress: 13/50 ✓ question_id=10_0
[independence_risk] Progres

...

[mult_div] Progress: 27/50 ✓ question_id=27_0
[mult_div] Progress: 28/50 ✓ question_id=30_0
[mult_div] Progress: 29/50 ✓ question_id=24_0
[mult_div] Progress: 30/50 ✓ question_id=32_0
[mult_div] Progress: 31/50 ✓ question_id=28_0
[mult_div] Progress: 32/50 ✓ question_id=33_0
[mult_div] Progress: 33/50 ✓ question_id=31_0
[mult_div] Progress: 34/50 ✓ question_id=34_0
[mult_div] Progress: 35/50 ✓ question_id=37_0
[mult_div] Progress: 36/50 ✓ question_id=35_0
[mult_div] Progress: 37/50 ✓ question_id=29_0
[mult_div] Progress: 38/50 ✓ question_id=38_0
[mult_div] Progress: 39/50 ✓ question_id=36_0
[mult_div] Progress: 40/50 ✓ question_id=40_0
[mult_div] Progress: 41/50 ✓ question_id=42_0
[mult_div] Progress: 42/50 ✓ question_id=39_0
[mult_div] Progress: 43/50 ✓ question_id=43_0
[mult_div] Progress: 44/50 ✓ question_id=44_0
[mult_div] Progress: 45/50 ✓ question_id=41_0
[mult_div] Progress: 46/50 ✓ question_id=45_0
[mult_div] Progress: 47/50 ✓ question_id=46_0
[mult_div] Progress: 48/50 ✓ quest

[dominant_strategies] Progress: 34/50 ✓ question_id=33_0
[dominant_strategies] Progress: 35/50 ✓ question_id=34_0
[dominant_strategies] Progress: 36/50 ✓ question_id=37_0
[dominant_strategies] Progress: 37/50 ✓ question_id=35_0
[dominant_strategies] Progress: 38/50 ✓ question_id=36_0
[dominant_strategies] Progress: 39/50 ✓ question_id=38_0
[dominant_strategies] Progress: 40/50 ✓ question_id=39_0
[dominant_strategies] Progress: 41/50 ✓ question_id=41_0
[dominant_strategies] Progress: 42/50 ✓ question_id=42_0
[dominant_strategies] Progress: 43/50 ✓ question_id=43_0
[dominant_strategies] Progress: 44/50 ✓ question_id=40_0
[dominant_strategies] Progress: 45/50 ✓ question_id=44_0
[dominant_strategies] Progress: 46/50 ✓ question_id=46_0
[dominant_strategies] Progress: 47/50 ✓ question_id=45_0
[dominant_strategies] Progress: 48/50 ✓ question_id=47_0
[dominant_strategies] Progress: 49/50 ✓ question_id=48_0
[dominant_strategies] Progress: 50/50 ✓ question_id=49_0
Saved CSV for q_type 'dominant_

[interpret_games] Progress: 41/50 ✓ question_id=41_0
[interpret_games] Progress: 42/50 ✓ question_id=42_0
[interpret_games] Progress: 43/50 ✓ question_id=40_0
[interpret_games] Progress: 44/50 ✓ question_id=44_0
[interpret_games] Progress: 45/50 ✓ question_id=43_0
[interpret_games] Progress: 46/50 ✓ question_id=48_0
[interpret_games] Progress: 47/50 ✓ question_id=46_0
[interpret_games] Progress: 48/50 ✓ question_id=45_0
[interpret_games] Progress: 49/50 ✓ question_id=47_0
[interpret_games] Progress: 50/50 ✓ question_id=49_0
Saved CSV for q_type 'interpret_games' to results_ds_7b/responses_interpret_games.csv
Processing file for q_type: backward_induction
[backward_induction] Progress: 1/50 ✓ question_id=51_0
[backward_induction] Progress: 2/50 ✓ question_id=50_0
[backward_induction] Progress: 3/50 ✓ question_id=54_0
[backward_induction] Progress: 4/50 ✓ question_id=53_0
[backward_induction] Progress: 5/50 ✓ question_id=52_0
[backward_induction] Progress: 6/50 ✓ question_id=58_0
[backwa

[add_sub] Progress: 45/50 ✓ question_id=46_0
[add_sub] Progress: 46/50 ✓ question_id=47_0
[add_sub] Progress: 47/50 ✓ question_id=44_0
[add_sub] Progress: 48/50 ✓ question_id=45_0
[add_sub] Progress: 49/50 ✓ question_id=49_0
[add_sub] Progress: 50/50 ✓ question_id=48_0
Saved CSV for q_type 'add_sub' to results_ds_7b/responses_add_sub.csv
Processing file for q_type: maximize_expected_utility
[maximize_expected_utility] Progress: 1/50 ✓ question_id=0_0
[maximize_expected_utility] Progress: 2/50 ✓ question_id=2_0
[maximize_expected_utility] Progress: 3/50 ✓ question_id=4_0
[maximize_expected_utility] Progress: 4/50 ✓ question_id=3_0
[maximize_expected_utility] Progress: 5/50 ✓ question_id=1_0
[maximize_expected_utility] Progress: 6/50 ✓ question_id=5_0
[maximize_expected_utility] Progress: 7/50 ✓ question_id=8_0
[maximize_expected_utility] Progress: 8/50 ✓ question_id=9_0
[maximize_expected_utility] Progress: 9/50 ✓ question_id=6_0
[maximize_expected_utility] Progress: 10/50 ✓ question_id

[sunk_cost] Progress: 14/50 ✓ question_id=10_0
[sunk_cost] Progress: 15/50 ✓ question_id=15_0
[sunk_cost] Progress: 16/50 ✓ question_id=17_0
[sunk_cost] Progress: 17/50 ✓ question_id=13_0
[sunk_cost] Progress: 18/50 ✓ question_id=18_0
[sunk_cost] Progress: 19/50 ✓ question_id=16_0
[sunk_cost] Progress: 20/50 ✓ question_id=21_0
[sunk_cost] Progress: 21/50 ✓ question_id=22_0
[sunk_cost] Progress: 22/50 ✓ question_id=19_0
[sunk_cost] Progress: 23/50 ✓ question_id=20_0
[sunk_cost] Progress: 24/50 ✓ question_id=23_0
[sunk_cost] Progress: 25/50 ✓ question_id=26_0
[sunk_cost] Progress: 26/50 ✓ question_id=27_0
[sunk_cost] Progress: 27/50 ✓ question_id=28_0
[sunk_cost] Progress: 28/50 ✓ question_id=29_0
[sunk_cost] Progress: 29/50 ✓ question_id=25_0
[sunk_cost] Progress: 30/50 ✓ question_id=24_0
[sunk_cost] Progress: 31/50 ✓ question_id=32_0
[sunk_cost] Progress: 32/50 ✓ question_id=30_0
[sunk_cost] Progress: 33/50 ✓ question_id=34_0
[sunk_cost] Progress: 34/50 ✓ question_id=35_0
[sunk_cost] P